In [68]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
% matplotlib inline

In [124]:
class Track:
    """
    Track consists of measured energy Edet and list of voxels' coordinates.
    The coordinates are the same for voxel weight and for voxel derivatives.
    Class Track has a gloabal pointer to Phantom and to the Phantom filled by derivatives.
    """
    phantom_w = None
    phantom_d = None
    Einc = None  # incident energy

    def __init__(self, Edet, voxels):
        self.Edet = Edet  # Energy detector meausurement for this event
        self.voxels = voxels[:]  # use copies of Voxels (for references assign voxels)

    def __repr__(self):
        s = 'Edet ' + str(Edet) + ' voxels'
        for voxel in self.voxels:
            s += ' ' + str(voxel)
        return s
    
    def loss(self):
        dE = self.Einc - self.Edet
        for voxel in self.voxels:
            row, col = voxel
            dE -= Track.phantom_w[row, col]
        return dE
    
    def loss_der(self):
        dE = self.Einc - self.Edet
        for voxel in self.voxels:
            row, col = voxel
            dE -= Track.phantom_w[row, col]

        # modify derivative for this voxel
        for voxel in self.voxels:
            Track.phantom_d[row, col] = -dE  # minus because our loss function is (y - h(w))
        return dE

    def print(self):
        print(self.Edet, end='\t')
        for row_col in self.voxels:
            row, col = row_col
            # print('Track.phantom_w[row, col] =', Track.phantom_w[row, col])
            # print(Track.phantom_w[row, col], end='\t')  # print voxel value
            print(row_col, end='\t')                      # print voxel coordinates
        print()

def print_phantom(phantom):
    for row in range(phantom.shape[0]):
        for col in range(phantom.shape[1]):
            print('{:8.2f}'.format(phantom[row, col]), end=' ')
        print()

N = 2
phantom_v = np.zeros((N, N))  # true values

phantom_w = np.zeros((N, N))  # weights
phantom_d = np.zeros((N, N))  # derivatives

# incident energy
Einc = 100  # MeV

#
# initialize Track global variables
#
Track.phantom_w = phantom_w
Track.phantom_d = phantom_d
Track.Einc = Einc

# set the phantom density (in terms of energy loss)
phantom_v[1, 1] = 10  # loss 10 MeV
phantom_v[1, 0] = 3   # loss 3 MeV
phantom_v[0, 0] = 5   # loss 5 MeV

print('"real" phantom\n')
print_phantom(phantom_v)

tracks = []
# the track energy will be measured later
tracks.append(Track(Einc, [(0, 0), (0, 1)]))
tracks.append(Track(Einc, [(1, 0), (1, 1)]))
tracks.append(Track(Einc, [(0, 0), (1, 0)]))
tracks.append(Track(Einc, [(0, 1), (1, 1)]))
tracks.append(Track(Einc, [(0, 0)]))
tracks.append(Track(Einc, [(1, 0), (0, 1)]))
tracks.append(Track(Einc, [(1, 1)]))
tracks.append(Track(Einc, [(1, 0)]))
tracks.append(Track(Einc, [(0, 0), (1, 1)]))
tracks.append(Track(Einc, [(0, 1)]))

# #############################################
# #
# # What will happen if I add some more tracks? -- Nothing bad, the same result as expected.
# #
# #############################################

# tracks.append(Track(Einc, [(1, 0), (1, 1)]))
# tracks.append(Track(Einc, [(0, 0), (1, 0)]))
# tracks.append(Track(Einc, [(0, 1), (1, 1)]))

# measure the energy
for track in tracks:
    Elost = 0
    for voxel in track.voxels:
        row, col = voxel
        Elost += phantom_v[row, col]
    track.Edet = Einc - Elost

print('\nMeasured tracks:')

for track in tracks:
    track.print()

"real" phantom

    5.00     0.00 
    3.00    10.00 

Measured tracks:
95.0	(0, 0)	(0, 1)	
87.0	(1, 0)	(1, 1)	
92.0	(0, 0)	(1, 0)	
90.0	(0, 1)	(1, 1)	
95.0	(0, 0)	
97.0	(1, 0)	(0, 1)	
90.0	(1, 1)	
97.0	(1, 0)	
85.0	(0, 0)	(1, 1)	
100.0	(0, 1)	


In [125]:
# Cost function
def costFunction(tracks, phantom_w, phantom_d):
    """
    Returns cost function.
    The derivatives phantom_d will be changed in-place as passed by reference
    """
    J = 0.  # cost function
    # clear derivatives
    phantom_d = np.zeros((phantom_d.shape[0], phantom_d.shape[1]))  # default dtype=float

    for track in tracks:
        loss = track.loss_der()  # calculates loss function J and modifies phantom_d
        J += loss * loss
    
    phantom_d /= len(tracks)
    return J / len(tracks) / 2


J = costFunction(tracks, phantom_w, phantom_d)
print('\nInitial Cost function J =', J)

print('\n"real" phantom\n')
print_phantom(phantom_v)

print('\nInitial "weights" phantom: should be empty\n')
print_phantom(phantom_w)

print('\n"derivatives" phantom\n')
print_phantom(phantom_d)

# introduce the correction
phantom_w -= phantom_d

print('\n"weights" phantom after correction\n')
print_phantom(phantom_w)

#
# the next iteration
#
print('\nThe next iteration')
print('------------------\n')

J = costFunction(tracks, phantom_w, phantom_d)
print('Cost function J =', J)

print('\n"weights" phantom\n')
print_phantom(phantom_w)

print('\n"derivatives" phantom\n')
print_phantom(phantom_d)

# introduce the correction
phantom_w -= phantom_d

print('\n"weights" phantom after correction\n')
print_phantom(phantom_w)


Initial Cost function J = 36.3

"real" phantom

    5.00     0.00 
    3.00    10.00 

Initial "weights" phantom: should be empty

    0.00     0.00 
    0.00     0.00 

"derivatives" phantom

   -5.00    -0.00 
   -3.00   -15.00 

"weights" phantom after correction

    5.00     0.00 
    3.00    15.00 

The next iteration
------------------

Cost function J = 5.0

"weights" phantom

    5.00     0.00 
    3.00    15.00 

"derivatives" phantom

   -0.00    -0.00 
   -0.00     5.00 

"weights" phantom after correction

    5.00     0.00 
    3.00    10.00 


In [106]:
# POSITIVE slope tracks -- uses transposition

N = 5
for row in range(N):
    i, j = row, 0
    for k in range(row+1):
        print((i-k, j+k), 'bottom:', (N-1-(j+k), N-1-(i-k)))
    print()

(0, 0) bottom: (4, 4)

(1, 0) bottom: (4, 3)
(0, 1) bottom: (3, 4)

(2, 0) bottom: (4, 2)
(1, 1) bottom: (3, 3)
(0, 2) bottom: (2, 4)

(3, 0) bottom: (4, 1)
(2, 1) bottom: (3, 2)
(1, 2) bottom: (2, 3)
(0, 3) bottom: (1, 4)

(4, 0) bottom: (4, 0)
(3, 1) bottom: (3, 1)
(2, 2) bottom: (2, 2)
(1, 3) bottom: (1, 3)
(0, 4) bottom: (0, 4)



In [107]:
# NEGATIVE slope tracks -- uses transposition

N = 5
for row in range(N):
    i, j = N - 1 - row, 0
    for k in range(row+1):
        print((i+k, j+k), 'top:', (j+k, i+k))
    print()

(4, 0) top: (0, 4)

(3, 0) top: (0, 3)
(4, 1) top: (1, 4)

(2, 0) top: (0, 2)
(3, 1) top: (1, 3)
(4, 2) top: (2, 4)

(1, 0) top: (0, 1)
(2, 1) top: (1, 2)
(3, 2) top: (2, 3)
(4, 3) top: (3, 4)

(0, 0) top: (0, 0)
(1, 1) top: (1, 1)
(2, 2) top: (2, 2)
(3, 3) top: (3, 3)
(4, 4) top: (4, 4)



In [123]:
"""NB that the second coordinate in POSITIVE and NEGATIVE pairs is the same.
The first one in NEGATIVE is related to first in POSITIVE as 4-x"""

N = 5

print('Generate positive slope tracks')
tracks_pos = []
for row in range(N):
    i, j = row, 0
    track_top = []
    track_bot = []
    for k in range(row+1):
        print((i-k, j+k), 'bottom:', (N-1-(j+k), N-1-(i-k)))
        track_top.append((i-k, j+k))
        track_bot.append((N-1-(j+k), N-1-(i-k)))
    print()
    tracks_pos.append(track_top)
    tracks_pos.append(track_bot)

# remove last track - it's a duplicate of previous
del tracks_pos[-1]

print('positive tracks:')
for track in tracks_pos:
    print(track)
print()

"""Create negative slope tracks changing the x-coordinate to (N-1) - x"""

tracks_neg = []
for track_pos in tracks_pos:
    track_neg = []
    for cell in track_pos:
        track_neg.append((N - 1 - cell[0], cell[1]))
    tracks_neg.append(track_neg)

print('positive tracks')
for track in tracks_pos:
    print(track)

print('negative tracks')
for track in tracks_neg:
    print(track)

'NB that the second coordinate in POSITIVE and NEGATIVE pairs is the same.\nThe first one in NEGATIVE is related to first in POSITIVE as 4-x'

Generate positive slope tracks
(0, 0) bottom: (4, 4)

(1, 0) bottom: (4, 3)
(0, 1) bottom: (3, 4)

(2, 0) bottom: (4, 2)
(1, 1) bottom: (3, 3)
(0, 2) bottom: (2, 4)

(3, 0) bottom: (4, 1)
(2, 1) bottom: (3, 2)
(1, 2) bottom: (2, 3)
(0, 3) bottom: (1, 4)

(4, 0) bottom: (4, 0)
(3, 1) bottom: (3, 1)
(2, 2) bottom: (2, 2)
(1, 3) bottom: (1, 3)
(0, 4) bottom: (0, 4)

positive tracks:
[(0, 0)]
[(4, 4)]
[(1, 0), (0, 1)]
[(4, 3), (3, 4)]
[(2, 0), (1, 1), (0, 2)]
[(4, 2), (3, 3), (2, 4)]
[(3, 0), (2, 1), (1, 2), (0, 3)]
[(4, 1), (3, 2), (2, 3), (1, 4)]
[(4, 0), (3, 1), (2, 2), (1, 3), (0, 4)]



'Create negative slope tracks changing the x-coordinate to (N-1) - x'

positive tracks
[(0, 0)]
[(4, 4)]
[(1, 0), (0, 1)]
[(4, 3), (3, 4)]
[(2, 0), (1, 1), (0, 2)]
[(4, 2), (3, 3), (2, 4)]
[(3, 0), (2, 1), (1, 2), (0, 3)]
[(4, 1), (3, 2), (2, 3), (1, 4)]
[(4, 0), (3, 1), (2, 2), (1, 3), (0, 4)]
negative tracks
[(4, 0)]
[(0, 4)]
[(3, 0), (4, 1)]
[(0, 3), (1, 4)]
[(2, 0), (3, 1), (4, 2)]
[(0, 2), (1, 3), (2, 4)]
[(1, 0), (2, 1), (3, 2), (4, 3)]
[(0, 1), (1, 2), (2, 3), (3, 4)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
